In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv('../input/train.csv')
X_train = train_df["question_text"].fillna("dieter").values
test_df = pd.read_csv('../input/test.csv')
X_test = test_df["question_text"].fillna("dieter").values
y = train_df["target"]

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, concatenate
from keras.layers import CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D
from keras.layers import Add, BatchNormalization, Activation, CuDNNLSTM, Dropout
from keras.layers import *
from keras.models import *
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import gc
from sklearn import metrics
from keras.optimizers import *

In [ ]:
maxlen = 70
max_features = 50000
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    TIME_STEPS = inputs.shape[1].value
    SINGLE_ATTENTION_VECTOR = False
    
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [ ]:
def noise_measurement(train, noise_level):
    noised_train = train.copy()
    to_transform = np.random.random(train.shape) < noise_level
    transform_y = np.random.randint(0, max_features, size=train.shape)
    noised_train[to_transform] = transform_y[to_transform]
    return noised_train

In [ ]:
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_1 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_1[i] = embedding_vector

del embeddings_index; gc.collect() 

In [ ]:
EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_2 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_2[i] = embedding_vector
del embeddings_index; gc.collect()

In [ ]:
EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix_3 = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix_3[i] = embedding_vector
        
del embeddings_index; gc.collect()   

In [ ]:
# # https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
# from gensim.models import KeyedVectors

# EMBEDDING_FILE = '../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
# embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# word_index = tokenizer.word_index
# nb_words = min(max_features, len(word_index))
# embedding_matrix_4 = (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
# for word, i in word_index.items():
#     if i >= max_features: continue
#     if word in embeddings_index:
#         embedding_vector = embeddings_index.get_vector(word)
#         embedding_matrix_4[i] = embedding_vector
        
# del embeddings_index; gc.collect()        

# Concatenating the embeddings

In [ ]:
embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2, embedding_matrix_3], axis=0)  
del embedding_matrix_1, embedding_matrix_2, embedding_matrix_3
gc.collect()
np.shape(embedding_matrix)

In [ ]:
from sklearn.model_selection import train_test_split
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y, test_size = 0.1, random_state=42)

# MODEL 1: Conv2D

In [ ]:
def model1():
    inp = Input(shape=(maxlen, ))
    embed = Embedding(max_features, embed_size * 1, weights=[embedding_matrix], trainable=False)(inp)
    embed = SpatialDropout1D(0.1)(embed)
    
    #x = Reshape((maxlen, embed_size * 3, 1))(embed)
    
    filter_sizes = [1,2,3,5]
    num_filters = 64
    
    conv_0 = Conv1D(num_filters, filter_sizes[0], padding='valid', kernel_initializer='normal', activation='relu')(embed)
    conv_1 = Conv1D(num_filters, filter_sizes[1], padding='valid', kernel_initializer='normal', activation='relu')(embed)
    conv_2 = Conv1D(num_filters, filter_sizes[2], padding='valid', kernel_initializer='normal', activation='relu')(embed)
    conv_3 = Conv1D(num_filters, filter_sizes[3], padding='valid', kernel_initializer='normal', activation='relu')(embed)

    maxpool_0 = MaxPool1D(pool_size=(maxlen - filter_sizes[0] + 1), strides=(1), padding='valid')(conv_0)
    maxpool_1 = MaxPool1D(pool_size=(maxlen - filter_sizes[1] + 1), strides=(1), padding='valid')(conv_1)
    maxpool_2 = MaxPool1D(pool_size=(maxlen - filter_sizes[2] + 1), strides=(1), padding='valid')(conv_2)
    maxpool_3 = MaxPool1D(pool_size=(maxlen - filter_sizes[3] + 1), strides=(1), padding='valid')(conv_3)

    concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])
    
    #gmp = GlobalMaxPooling1D()(concatenated_tensor)
    #gap = GlobalAveragePooling1D()(concatenated_tensor)
    
    #conc = Concatenate(axis=1)([gmp, gap])
    
    flatten = Flatten()(concatenated_tensor)
    
    x = flatten
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    outp = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=1e-3, decay=1e-6),
                  metrics=['accuracy'])    

    return model

In [ ]:
MODEL1 = model1()
MODEL1.summary()

batch_size = 2048
epochs = 4

hist = MODEL1.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
MODEL1.save('./model1.h5')

In [ ]:
pred_val_y_1 = MODEL1.predict([X_val], batch_size=1024, verbose=1)
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(y_val, (pred_val_y_1 > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh_1 = thresholds[0][0]
print("Best threshold: ", best_thresh_1)

y_pred_1 = MODEL1.predict(x_test, batch_size=1024, verbose=True)

# MODEL 2: LSTM

In [ ]:
def model2():
    inp = Input(shape=(maxlen, ))
    embed = Embedding(max_features, embed_size * 1, weights=[embedding_matrix], trainable=False)(inp)
    x = embed
    x = SpatialDropout1D(0.1)(x)
    
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = attention_3d_block(x)
    x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
    x = AttLayer(maxlen)(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    outp = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=1e-3, decay=1e-6),
                  metrics=['accuracy'])    

    return model

In [ ]:
MODEL2 = model2()
MODEL2.summary()

batch_size = 2048
epochs = 5

hist = MODEL2.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
MODEL2.save('./model2.h5')

In [ ]:
pred_val_y_2 = MODEL2.predict([X_val], batch_size=1024, verbose=1)
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(y_val, (pred_val_y_2 > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh_2 = thresholds[0][0]
print("Best threshold: ", best_thresh_2)

y_pred_2 = MODEL2.predict(x_test, batch_size=1024, verbose=True)

# MODEL 3: Conv1D

In [ ]:
def model3():
    filters = 64
    
    inp = Input(shape=(maxlen, ))
    embed = Embedding(max_features, embed_size * 1, weights=[embedding_matrix], trainable=False)(inp)
    x = embed
    x = SpatialDropout1D(0.1)(x)
    
    x = Conv1D(filters, 1, activation='relu', padding='valid', kernel_initializer='normal')(x)
    x = Dropout(0.1)(x)
    
    x = Conv1D(filters, 2, activation='relu', padding='valid', kernel_initializer='normal')(x)
    x = Dropout(0.1)(x)
    
    x = Conv1D(filters, 3, activation='relu', padding='valid', kernel_initializer='normal')(x)
    x = Dropout(0.1)(x)
    
    x = Conv1D(filters, 5, activation='relu', padding='valid', kernel_initializer='normal')(x)
    x = Dropout(0.1)(x)
    
    x = Flatten()(x)
    #x = GlobalAveragePooling1D()(x)
    
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    outp = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=1e-3, decay=1e-6),
                  metrics=['accuracy'])    

    return model

In [ ]:
MODEL3 = model3()
MODEL3.summary()

batch_size = 2048
epochs = 10

hist = MODEL3.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
MODEL3.save('./model3.h5')

In [ ]:
pred_val_y_3 = MODEL3.predict([X_val], batch_size=1024, verbose=1)
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(y_val, (pred_val_y_3 > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh_3 = thresholds[0][0]
print("Best threshold: ", best_thresh_3)

y_pred_3 = MODEL3.predict(x_test, batch_size=1024, verbose=True)

# MODEL 4: GRU

In [ ]:
def model4():
    inp = Input(shape=(maxlen, ))
    embed = Embedding(max_features, embed_size * 1, weights=[embedding_matrix], trainable=False)(inp)
    x = embed
    x = SpatialDropout1D(0.1)(x)
    
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = attention_3d_block(x)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = AttLayer(64)(x)
    
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    outp = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=1e-3, decay=1e-6),
                  metrics=['accuracy'])    

    return model

In [ ]:
MODEL4 = model4()
MODEL4.summary()

batch_size = 2048
epochs = 5

hist = MODEL4.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), verbose=True)
MODEL4.save('./model4.h5')

In [ ]:
pred_val_y_4 = MODEL4.predict([X_val], batch_size=1024, verbose=1)
thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(y_val, (pred_val_y_4 > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh_4 = thresholds[0][0]
print("Best threshold: ", best_thresh_4)

y_pred_4 = MODEL4.predict(x_test, batch_size=1024, verbose=True)

# Concat Result & Best Threshold

In [ ]:
pred_val_y = (2.5*pred_val_y_1 + 2.5*pred_val_y_2 + 2.5*pred_val_y_3 + 2.5*pred_val_y_4)/10

thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(y_val, (pred_val_y > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
print("Best threshold: ", best_thresh)

# Submission File

In [ ]:
y_pred = (2.5*y_pred_1 + 2.5*y_pred_2 + 2.5*y_pred_3 + 2.5*y_pred_4)/10
y_te = (y_pred[:,0] > best_thresh).astype(np.int)

submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("submission.csv", index=False)

In [ ]:
from IPython.display import HTML
import base64  
import pandas as pd  

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv(index =False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submit_df)